<img src="logo.png">

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn import datasets

Para este ejemplo vamos a utilizar el [Wisconsin Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)). Es un dataset de imagenes de celulas obtenidas de analisis de personas que sufren un posible cancer de mama.

Las imagenes tienen el siguiente aspecto:

![image.png](ml12.png)

In [ ]:
cancer_datos = datasets.load_breast_cancer()
cancer_datos.keys()

In [ ]:
print(cancer_datos["DESCR"])

In [ ]:
cancer_datos["target_names"]

In [ ]:
cancer_df = pd.DataFrame(cancer_datos["data"],
                           columns=cancer_datos["feature_names"]
                          )

cancer_df["objetivo"] = cancer_datos["target"]

El dataset contiene los valores medios de ciertos parametros del núcleo de las celulas mostradas en las imágenes, así como dichos valores para la celula con características más preocupantes

In [ ]:
cancer_df.shape

In [ ]:
cancer_df.head()

In [ ]:
cancer_df["objetivo"].value_counts(True)

In [ ]:
cancer_df["objetivo"] = cancer_df["objetivo"].replace({1:0,0:1})

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
train_df, test_df = train_test_split(cancer_df, test_size=0.4)

variables_entrenamiento = cancer_datos["feature_names"]
variable_objetivo = "objetivo"

In [ ]:
columna_entrenamiento = "worst area"

plt.plot(train_df[columna_entrenamiento], train_df.objetivo, '.r')
plt.xlabel("Peor area encontrada en los nucleos de la imagen")
plt.ylabel("Diagnostico (Maligno|Beningno)");

In [ ]:
modelo_ols = LinearRegression()

modelo_ols.fit(train_df[[columna_entrenamiento]],
               train_df[variable_objetivo])

predicciones = modelo_ols.predict(test_df[[columna_entrenamiento]])

predicciones[:10]

In [ ]:
plt.plot(test_df[columna_entrenamiento], test_df.objetivo, '.r')
plt.plot(test_df[columna_entrenamiento], predicciones, '.b')
plt.xlabel("Peor area encontrada en los nucleos de la imagen")
plt.ylabel("Diagnostico (Maligno|Beningno)")

In [ ]:
def funcion_logistica(x,L=1,k=1,x0=0):
    return L / (1+np.exp(-k*(x-x0)))

predicciones_probabilidades = list(map(funcion_logistica,predicciones))

In [ ]:
plt.plot(test_df[columna_entrenamiento], test_df.objetivo, '.r')
plt.plot(test_df[columna_entrenamiento], predicciones, '.b')
plt.plot(test_df[columna_entrenamiento], predicciones_probabilidades,'.g')

plt.xlabel("Peor area encontrada en los nucleos de la imagen")
plt.ylabel("Diagnostico (Maligno|Beningno)")

In [ ]:
from functools import partial

funcion_logit_k5 = partial(funcion_logistica, k=5)

predicciones_probabilidades = list(map(funcion_logit_k5, predicciones))

plt.plot(test_df[columna_entrenamiento], test_df.objetivo, '.r')
plt.plot(test_df[columna_entrenamiento], predicciones, '.b')
plt.plot(test_df[columna_entrenamiento], predicciones_probabilidades,'.g')

plt.xlabel("Peor area encontrada en los nucleos de la imagen")
plt.ylabel("Diagnostico (Maligno|Beningno)");

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogisticRegression??

In [ ]:
X = cancer_df[variables_entrenamiento]
y = cancer_df[variable_objetivo]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_train

In [ ]:
clf = LogisticRegression(solver="liblinear")
clf.fit(X_train, y_train)
predicciones = clf.predict(X_test)

predicciones[:10]

In [ ]:
predicciones_probabilidades = clf.predict_proba(X_test)
predicciones_probabilidades[:10]

In [ ]:
plt.hist(predicciones_probabilidades)

In [ ]:
probs_df = pd.DataFrame(predicciones_probabilidades)

In [ ]:
X = X_test.reset_index().copy()
X["objetivo"] = y_test.tolist()
X["prediccion"] = predicciones
X = pd.concat([X, probs_df], axis=1)
X[["objetivo", "prediccion", 0, 1]].head(20)